In [94]:
import json

import pickle
import numpy as np

In [95]:
def load_data_json(json_path):
    f = open(json_path, 'r', encoding='utf-8')
    json_data = json.load(f)
    f.close()
    return json_data

In [96]:
mode = 'public'
json_path = f'./data/{mode}_complete.json'
json_data = load_data_json(json_path)

results_path = f'./output_results/public_result_colbert_neg100_epoch2'
with open(results_path, 'rb') as handle:
    results = pickle.load(handle)
            

In [97]:
binary_result_path  = f'./output_results/public_binary_result_cross_encoder_binary_neg60_alldata_epoch2'
with open(binary_result_path, 'rb') as handle:
    binary_results = pickle.load(handle)
    
print(len(binary_results))

421


In [98]:
print(len(json_data))

421


In [99]:
did2data = {}
for d in json_data:
    did = d['did']
    did2data[did] = d


In [100]:
from gensim.summarization import bm25
corpus_ids = [d['did'] for d in json_data]
corpus = []
titles = []
title_and_corpus = []

from zhon.hanzi import punctuation
import string
for i, d in enumerate(json_data):
    text_words = d['replaced_text_sentence'].split()
    title_words = d['replaced_title_sentence'].split()

    filter_title_words = [w for w in title_words if w not in punctuation and w not in string.punctuation]

    filter_text_words = [w for w in text_words if w not in punctuation and w not in string.punctuation]

    corpus.append(filter_text_words)
    titles.append(filter_title_words)
    title_and_corpus.append(filter_title_words + filter_text_words)

bm25Model = bm25.BM25(title_and_corpus)

In [101]:
for i, query in enumerate((title_and_corpus)):
    q_did = corpus_ids[i]
    result = results[q_did]
#     bm25_scores = []
    scores = bm25Model.get_scores(query)
    for idx, score in enumerate(scores):
        ref_did = corpus_ids[idx]
        if q_did == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[q_did] = result


In [102]:
import pandas as pd
import glob
# files = glob.glob("./data/Keywords/*.xlsx")
files = glob.glob("./data/Keywords/02pest.list.xlsx")
same_words_dict = {}

other_keywords = [
    '葡萄露菌病',
    '白葉枯病'
    '舞蛾',
]

for file in files:
    df = pd.read_excel(file, header=None)
    df_list = df.values.tolist()
    for words in df_list:
        change_to = words[0]
        same_words_dict[change_to] = change_to
        for word in words:
            word = str(word)
            if word == 'nan':
                break
            if word != change_to:
                same_words_dict[word] = change_to

for other_keyword in other_keywords:
    same_words_dict[other_keyword] = other_keyword

In [103]:
# keyword_sets = []
for i, d in enumerate(json_data):
    did = d['did']
    text_words = d['text_sentence'].split()
    title_words = d['title_sentence'].split()
    all_words = text_words + title_words
    keyword_set = set()
    for k, v in same_words_dict.items():
        if k in all_words:
            keyword_set.add(same_words_dict[k])
#     text = ' '.join(all_words)
#     key_phrases = jio.keyphrase.extract_keyphrase(text)
#     for key_phrase in key_phrases:
#         if key_phrase in same_words_dict:
#             keyword_set.add(same_words_dict[key_phrase])
#         else:
#             keyword_set.add(key_phrase)
#     keyword_sets.append(keyword_set)
    json_data[i]['keyword_set'] = keyword_set

In [104]:
def jaccard_score(set1, set2):
    u = set1.union(set2) 
    i = set1.intersection(set2)
    if len(u) == 0:
        return 0
    return len(i) / len(u)

In [105]:
from collections import defaultdict
keyword_set_sim = defaultdict(dict)
for i in range(len(json_data)):
    for k in range(len(json_data)):
        
        did_i = json_data[i]['did']
        did_k = json_data[k]['did']
        
        set_i = json_data[i]['keyword_set']
        set_k = json_data[k]['keyword_set']
        
        sim_socre = jaccard_score(set_i, set_k)
        keyword_set_sim[did_i][did_k] = sim_socre

for i in range(len(json_data)):
    did_i = json_data[i]['did']
    result = results[did_i]
    set_sim_score = keyword_set_sim[did_i]
    for ref_did, score in set_sim_score.items():
        if did_i == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[did_i] = result

In [115]:
topk = 20

pos_pairs = []

second_num_of_std = 2

num_of_std = 1.5
binary_th = 0.1
set_th = 0.2

for k, values in results.items():
    did = str(k)
    
    binary_value = binary_results[k]
    all_ref_ids = np.array([v[0] for v in values])
    data = did2data[did]
    pos_dids = data['pos_dids']
        
    print(f'did = {k}')
    
#     bm25_pos_dids = data['bm25_pos_dids'][:topk]
    bm25_scores = [v[3] for v in values]
    bm25_scores = np.array(bm25_scores)
    bm25_mean = np.mean(bm25_scores)
    bm25_median = np.median(bm25_scores)
    bm25_std = np.std(bm25_scores)
    bm25_max_score = np.max(bm25_scores)
    bm25_th = bm25_median + num_of_std * bm25_std
    bm25_pos_idx = bm25_scores >= bm25_th
    bm25_preds = all_ref_ids[bm25_pos_idx]
    
    
    
    set_scores =  [v[4] for v in values]
    set_scores = np.array(set_scores)
    set_pos_idx = set_scores >= set_th
    set_preds = all_ref_ids[set_pos_idx]
    set_pred_scores = set_scores[set_pos_idx]
    
    
    
    

    scores = [v[2] for v in values]
    scores = np.array(scores)
    mean = np.mean(scores)
    median = np.median(scores)
    std = np.std(scores)
    th = median + num_of_std * std
    pos_idx = scores > th
    pos_ref_ids = all_ref_ids[pos_idx]
    pos_scores = scores[pos_idx]

    
    binary_scores = [v[1] for v in binary_value]
    binary_scores = np.array(binary_scores)
    binary_pos_idx = binary_scores >= binary_th
    binary_preds = all_ref_ids[binary_pos_idx]
    binary_pos_scores = binary_scores[binary_pos_idx]
    
    
    mix_preds = list(set(bm25_preds) | set(set_preds))
    mix_preds = list(set(mix_preds) & set(binary_preds))
    print(mix_preds)

#     print('score based predict topk did and score')

#     print(pos_ref_ids)
#     print(pos_scores)
    
    secnod_pred_mean = np.mean(pos_scores)
    second_median = np.median(pos_scores)
    second_std = np.std(pos_scores)
    second_th = second_median + second_num_of_std * second_std
    second_pos_idx = pos_scores >=  second_th
    second_preds = pos_ref_ids[second_pos_idx]
    second_pos_scores = pos_scores[second_pos_idx]
#     print('second pred')
#     print(second_preds)
#     print(second_pos_scores)

    preds = mix_preds
    
    for pred in preds:
        pos_pairs.append((did, pred))

    


did = 7
[]
did = 13
['134', '265', '234', '1028', '190', '257', '249', '1070', '1085', '92', '160', '42', '56', '416', '244', '410', '168']
did = 19
['447', '564', '1064', '538', '545', '390', '494', '398', '593', '198', '315', '461', '30', '60', '87', '397', '288', '822', '844', '889', '1040', '451', '867', '294', '996', '1019', '433', '1034', '915', '79', '172', '660', '685', '426', '195', '452', '102', '409', '352', '96', '749', '1091', '1072', '399', '417', '960', '525', '886', '1131', '460', '705', '1022', '1024', '1001', '1360']
did = 20
['489', '499']
did = 24
['50', '31', '69']
did = 28
['592', '1061', '647', '1064', '494', '975', '388', '30', '202', '38', '1009', '464', '714', '450', '213', '359', '761', '854', '71', '580', '1140']
did = 30
['1064', '38', '464', '71', '450', '494', '28']
did = 31
['50', '69']
did = 38
['592', '1061', '647', '1064', '494', '28', '975', '388', '30', '202', '1009', '464', '714', '450', '213', '359', '761', '854', '71', '580', '1140']
did = 42
['1

['929', '491', '570', '820', '1098', '999']
did = 473
['909', '503', '903', '930', '1050', '834', '1051', '939']
did = 477
['901', '200', '503', '84', '202', '31', '685', '1330', '50', '1051', '401', '705', '264', '250', '254']
did = 485
['881', '45', '293', '617', '1105', '1330', '573', '50', '1009', '714', '389', '943', '128', '766', '666', '917', '1328', '31', '668', '1121', '69', '450', '718', '909', '488', '574', '358', '214']
did = 488
['881', '45', '617', '1105', '1330', '50', '1009', '485', '714', '389', '943', '128', '766', '666', '917', '903', '31', '1328', '668', '1121', '718', '909', '358']
did = 489
['20', '499']
did = 490
['1032', '175', '814', '908', '717', '80', '1182', '1029', '1080', '937', '615', '459', '965', '815', '935', '813', '818', '1399', '1094', '669', '407', '990', '991', '970', '1036']
did = 491
['469', '1098', '820', '570']
did = 494
['30', '1064', '38', '464', '71', '450', '28']
did = 497
['229', '803', '1048', '535', '247']
did = 499
['24', '489', '31', 

['447', '1064', '538', '413', '198', '269', '315', '288', '822', '1040', '1034', '451', '294', '996', '172', '426', '435', '409', '1072', '417', '960', '886', '1024', '1001']
did = 1124
['439', '528']
did = 1129
[]
did = 1131
['447', '861', '880', '564', '545', '390', '494', '398', '593', '315', '30', '397', '87', '822', '1040', '889', '1089', '451', '1019', '1034', '731', '433', '915', '172', '660', '685', '195', '452', '581', '125', '102', '409', '352', '96', '450', '19', '749', '1091', '417', '525', '460', '705', '1022', '399', '1360']
did = 1135
[]
did = 1137
['1092', '640', '880', '712', '1383', '109', '734', '941', '459', '818']
did = 1140
['975', '647', '30', '1064', '761', '38', '71', '450', '494', '28']
did = 1143
[]
did = 1156
['1368']
did = 1158
[]
did = 1162
[]
did = 1163
['159', '161', '145']
did = 1164
[]
did = 1175
['1307', '1306', '1231', '1313', '1251', '1245', '1303']
did = 1177
['1345', '1188', '1285', '1201']
did = 1182
['1399', '880', '815', '1080', '669', '175', '

In [116]:
same_words_dict['葉稻熱病']

'葉稻熱病'

In [117]:
len(pos_pairs)

4233

In [118]:
did2data['581']

{'did': '581',
 'title': 'ㄧ期作水稻進入分蘗期\u3000請注意葉稻熱病的防治',
 'text': '行政院農業委員會台東區農業改良場新聞稿ㄧ期作水稻進入分蘗期請注意葉稻熱病的防治今年開春以來天氣一直不太穩定，初期的低溫環境影響水稻植株的生長，目前氣溫雖已逐漸回升，惟伴隨著陰雨環境，極易造成病蟲害的孳生與蔓延。今年一期作水稻已進入分蘗期及分蘗盛期，經調查發現臺東縣局部地區，尤其靠近山邊之稻田，其中水稻葉色較為濃綠的稻田已陸續出現葉稻熱病初期的病斑，農友們應提早做好防治工作，以確保水稻的正常生長。水稻在低溫環境生長勢較慢，農友們往往增加施肥量以促進其生長，然而多施氮肥將造成植株徒長，葉片柔弱，降低對病害之抵抗，增加水稻罹患稻熱病的機會，不可偏施氮肥，以免加劇病害發生。葉稻熱病發病初期葉面形成褐色或暗綠色小斑點，如環境適合，擴大成紡綞形，此時病斑周圍呈黃色，中間為赤褐色，內部灰白色。嚴重時葉片枯萎甚至全株枯死。若發現水稻植株出現稻熱病病徵時，必須儘早做好防治工作；葉稻熱病之防治推薦的藥劑很多，可選擇下列其中一種來防治，如75％三賽唑可濕性粉劑，每公頃施用0.33公斤，稀釋3000倍；54％保米熱斯可濕性粉劑，每公頃施用0.66公斤，稀釋1500倍；43％嘉賜貝芬混合可濕性粉劑，每公頃施用1公斤，稀釋1000倍等，或參考植物保護手冊所列該病之推薦防治藥劑(www.tactri.gov.tw/htdocs/ppmtable/#catl)。勤加巡視稻田，審慎施用肥料，做好預防工作，期望本期作水稻免於稻熱病之危害。',
 'title_sentence': 'ㄧ期 作 水稻 進入 分蘗期 \u3000 請 注意 葉稻熱病 的 防治',
 'text_sentence': '行政院 農業 委員會 台東區 農業 改良場 新聞稿 ㄧ期 作 水稻 進入 分蘗 期請 注意 葉稻熱病 的 防治 今年 開春 以來 天氣 一直 不 太 穩定 ， 初期 的 低溫 環境 影響 水稻 植株 的 生長 ， 目前 氣溫 雖 已 逐漸 回升 ， 惟 伴隨 著 陰雨 環境 ， 極 易 造成 病蟲害 的 孳生 與 蔓延 。 今年 一 期 作 水稻 已 進入 分蘗期 及 分蘗 盛期 ， 經 調查 發現 臺東縣 局部 地區 ， 尤其 靠近 山 邊 之 稻田 ， 其中 水稻 葉色 較為 

In [119]:
import pandas as pd
submit_path = './submit/mix_binary_05_bm25_03.csv'
df = pd.DataFrame(pos_pairs, columns=['Test','Reference'])
df.to_csv(submit_path, index=None)